# Notebook to demonstrate the climate tools available
#  Based on data retreived via the cdsapi, you will need a cdsapi key to download the data

In [ ]:
# Imports
import os
import xarray as xr
import cdsapi

import tyto

In [ ]:
# Create directories 
if not os.path.isdir('test_data'): os.mkdir('test_data')

# set constants
YEARS = [2015]

In [ ]:
# Download 5 years of hourly ERA5 data for europe
c = cdsapi.Client()
out_fnames = []
for year in YEARS:
    out_fnames += [f"test_data/era5_temperature_europe_{year}.grib"]
    if not os.path.isfile(out_fnames[-1]):
        c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis',
                'variable': '2m_temperature',
                'year': year,
                'month': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                ],
                'day': [
                    '01', '02', '03',
                    '04', '05', '06',
                    '07', '08', '09',
                    '10', '11', '12',
                    '13', '14', '15',
                    '16', '17', '18',
                    '19', '20', '21',
                    '22', '23', '24',
                    '25', '26', '27',
                    '28', '29', '30',
                    '31',
                ],
                'time': [
                    '00:00', '06:00', '12:00', '18:00',
                ],
                'area': [
                    80, -10, 30, 60,
                ],
                'format': 'grib',
            },
        ).download(target=out_fnames[-1])



In [ ]:
# Open data in xarray
t2m_data_in = xr.open_mfdataset(out_fnames)
t2m_data = t2m_data_in.isel(latitude=100,longitude=100)
t2m_data

In [ ]:
t2m_data.t2m.plot()

In [ ]:
t2m_rolling = tyto.aggregate.rolling_reduce(
    t2m_data, time=50, center=True, how_dropna='all'
)
t2m_rolling

In [ ]:
t2m_data.t2m.plot()
t2m_rolling.t2m.plot()

In [ ]:
tyto.aggregate.rolling_reduce(
    t2m_data_in, time=30, lat=5, lon=5, center=True,
    how_dropna='all'
)